In [ ]:
import os
import zipfile
from tensorflow.keras import layers, models
from kaggle.api.kaggle_api_extended import KaggleApi

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

In [ ]:

def download_and_extract_dataset():
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files('shaunthesheep/microsoft-catsvsdogs-dataset', path='datasets', unzip=True)

    with zipfile.ZipFile('datasets/microsoft-catsvsdogs-dataset.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets/cats_vs_dogs')

def prepare_data():
    base_dir = 'datasets/cats_vs_dogs'
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')

    train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        subset='validation'
    )

    return train_generator, validation_generator

def build_and_train_cnn(train_generator, validation_generator):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=50
    )

    return model, history

def build_and_train_dnn(train_generator, validation_generator):
    model = models.Sequential([
        layers.Flatten(input_shape=(150, 150, 3)),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs=15,
        validation_data=validation_generator,
        validation_steps=50
    )

    return model, history

download_and_extract_dataset()
# train_generator, validation_generator = prepare_data()
# cnn_model, cnn_history = build_and_train_cnn(train_generator, validation_generator)
# dnn_model, dnn_history = build_and_train_dnn(train_generator, validation_generator)


NameError: name 'KaggleApi' is not defined